# utility

In [ ]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random

# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


# Torchvision provides lots of useful utilities for image preprocessing
# data wrapping as well as data augmentation.
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.RandomRotation(180),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.ToTensor(),
])

# 生成多张图片的方法
def spec_transformer(method):
    tfm = transforms.Compose([
        transforms.Resize((128, 128)),
        method,
        transforms.ToTensor(),
    ])
    return tfm

# 生成多张图片的方法 传入多个变换方法
def spec_transformers(methods):
    tfm = transforms.Compose([
        *[x for x in methods],
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ])
    return tfm

# 这个方法可以生成多张变换的图片
#     fff = [spec_transformers([]),
#            spec_transformers([transforms.RandomHorizontalFlip(1), transforms.RandomVerticalFlip(1)])]
# x = spec_transformers([])(im)


# parameters

In [ ]:
_exp_name = "main_aug"
batch_size = 64
_dataset_dir = "/kaggle/input/ml2022spring-hw3b/food11"
myseed = 76553366  # set a random seed for reproducibility
# The number of training epochs and patience.
n_epochs = 100
patience = 20 # If no improvement in 'patience' epochs, early stop
learning_rate = 3*1e-4
weight_decay = 1e-5
# 结果加权的向量 注意长度必须能被batch_size整除 并且等于图片复制的数量
ensemble_weight = np.array([0.4, 0.2, 0.2, 0.2])
# my_trans = [spec_transformers([transforms.RandAugment()]),
#             spec_transformers([transforms.RandomRotation(180),
#                                transforms.RandomResizedCrop((128, 128), scale=(0.5, 1.0), ratio=(0.8, 1.2)),
#                                transforms.RandomAutocontrast(),
#                                transforms.ColorJitter(brightness=(0.7, 1.3), contrast=(0.7, 1.3),
#                                                       saturation=(0.7, 1.3)),
#                                transforms.RandomPerspective(distortion_scale=np.random.rand())]),
#             spec_transformers([transforms.RandomHorizontalFlip(),
#                                transforms.RandomVerticalFlip(),
#                                transforms.RandomResizedCrop((128, 128), scale=(0.5, 1.0), ratio=(0.8, 1.2)),
#                                transforms.RandomAdjustSharpness(2 * np.random.rand()),
#                                transforms.ColorJitter(brightness=(0.7, 1.3), contrast=(0.7, 1.3),
#                                                       saturation=(0.7, 1.3))]),
#             spec_transformers([transforms.RandomHorizontalFlip(),
#                                transforms.RandomVerticalFlip(),
#                                transforms.RandomAffine(degrees=30, translate=(0.1, 0.2), shear=(0, 30)),
#                                transforms.RandomAdjustSharpness(2 * np.random.rand()),
#                                transforms.ColorJitter(brightness=(0.7, 1.3), contrast=(0.7, 1.3),
#                                                       saturation=(0.7, 1.3))])
#             ]

my_trans = [spec_transformers([transforms.RandAugment(),transforms.RandAugment(),transforms.RandAugment()]),
             spec_transformers([transforms.RandAugment(),transforms.RandomRotation(180),
                             transforms.RandomResizedCrop((128, 128), scale=(0.5, 1.0), ratio=(0.8, 1.2)),
                             transforms.RandomAutocontrast(),
                             transforms.ColorJitter(brightness=(0.7, 1.3), contrast=(0.7, 1.3),
                                                    saturation=(0.7, 1.3)),
                             transforms.RandomPerspective(distortion_scale=np.random.rand())]),
             spec_transformers([transforms.RandAugment(),transforms.RandomHorizontalFlip(),
                             transforms.RandomVerticalFlip(),
                             transforms.RandomResizedCrop((128, 128), scale=(0.5, 1.0), ratio=(0.8, 1.2)),
                             transforms.RandomAdjustSharpness(2*np.random.rand()),
                             transforms.ColorJitter(brightness=(0.7, 1.3), contrast=(0.7, 1.3),
                                                    saturation=(0.7, 1.3))]),
             spec_transformers([transforms.RandAugment(),transforms.RandomHorizontalFlip(),
                             transforms.RandomVerticalFlip(),
                             transforms.RandomAffine(degrees=30,translate=(0.1,0.2),shear=(0,30)),
                             transforms.RandomAdjustSharpness(2*np.random.rand()),
                             transforms.ColorJitter(brightness=(0.7, 1.3), contrast=(0.7, 1.3),
                                                    saturation=(0.7, 1.3))])
             ]

test_trans = [spec_transformers([])]+my_trans[1:]

num_of_transformers = len(my_trans)

# Data load

In [ ]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label
    
# 重写数据载入  tfms 为transformers的集合
class MultiFoodDataset(Dataset):

    def __init__(self, path, tfms, files=None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path, x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files is not None:
            self.files = files
        print(f"One {path} sample", self.files[0])
        self.transforms = tfms
        # 图片重复次数
        self.pic_duplicate_times = len(tfms)

    def __len__(self):
        return self.pic_duplicate_times * len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx // self.pic_duplicate_times]
        im = Image.open(fname)
        im = self.transforms[idx % self.pic_duplicate_times](im)
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1  # test has no label
        return im, label
    
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
same_seeds(myseed)
train_set = MultiFoodDataset(os.path.join(_dataset_dir,"training"), tfms=my_trans)
# train_set = FoodDataset(os.path.join(_dataset_dir,"training"), tfm=train_tfm)
print('训练数据集',len(train_set))
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = FoodDataset(os.path.join(_dataset_dir,"validation"), tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True) 
del train_set,valid_set

# Model

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)
    
class ResNet(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
        )
        self.fc = nn.Sequential(
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

# Train

In [ ]:
# Initialize a model, and put it on the device specified.
# model = Classifier().to(device)
model = models.resnet50(pretrained=False)
model.conv1.kernel_size = (3,3)
model.fc.out_features = 11
model.load_state_dict(torch.load(f"../input/model/aug_best.ckpt"))
model = model.to(device)

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay) 

# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0

for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        #print(imgs.shape,labels.shape)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # update logs
    if valid_acc > best_acc:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break

# Result

In [ ]:
# del train_loader,valid_loader
# test_set = FoodDataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
# test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# model_best = Classifier().to(device)
# model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))
# model_best.eval()
# prediction = []
# with torch.no_grad():
#     for data,_ in test_loader:
#         test_pred = model_best(data.to(device))
#         test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
#         prediction += test_label.squeeze().tolist()
        
# #create test csv
# def pad4(i):
#     return "0"*(4-len(str(i)))+str(i)
# df = pd.DataFrame()
# df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
# df["Category"] = prediction
# df.to_csv("submission.csv",index = False)        

# Result with Ensemble

In [ ]:
test_set = MultiFoodDataset(os.path.join(_dataset_dir, "test"), tfms=test_trans)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)


# 这里要加 , map_location='cpu'  因为GPU训练的模型转CPU会不兼容  这里用的参数 为kaggle 前几个版本的初始sample
model_best = models.resnet50(pretrained=False)
model_best.conv1.kernel_size = (3,3)
model_best.fc.out_features = 11
model_best.load_state_dict(torch.load(f"../input/model/main_aug_best.ckpt"))
# model_best.load_state_dict(torch.load(f"{_exp_name}_best.ckpt"))

model_best = model_best.to(device)

model_best.eval()
prediction = []


with torch.no_grad():
    for data, _ in test_loader:
        # shape 为batch_size * num_of_classes
        test_pred = model_best(data.to(device))
#         print(test_pred.shape)
        batch_len = test_pred.shape[0]
        # shape 为batch_size
        oh = test_pred.cpu().data.numpy().reshape(batch_len // num_of_transformers, num_of_transformers, 1000)
        oh = np.dot(ensemble_weight, oh)
        test_label = np.argmax(oh, axis=1)
        prediction += test_label.squeeze().tolist()


# create test csv
def pad4(i):
    return "0" * (4 - len(str(i))) + str(i)


df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1, len(test_set) // num_of_transformers + 1)]
df["Category"] = prediction
df.to_csv("submission.csv", index=False)